In [ ]:
from huggingface_hub import login

login()

In [1]:
from datasets import load_dataset

dataset = load_dataset("norhsangpha/shn-asr-datasets")


In [2]:
train_dataset = dataset['train']
eval_dataset = dataset['test']

In [3]:
from transformers import VitsTokenizer, VitsModel

# Load the VITS MMS TTS tokenizer
tokenizer = VitsTokenizer.from_pretrained("./Finetune/finetune-hf-vits/tokenizer/shn-add-token")
model = VitsModel.from_pretrained("./Finetune/finetune-hf-vits/models/mms-tts-shn-checkpoint")

# Extend the tokenizer's vocabulary with the additional characters
new_tokens = ["ၷ", "ၹ", "ၻ", "ၾ", "ႊ"]
# tokenizer.add_tokens(new_tokens)

c:\Users\NoerN\.conda\envs\ttslabs\lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\NoerN\.conda\envs\ttslabs\lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\NoerN\.conda\envs\ttslabs\lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [4]:
print(tokenizer)

VitsTokenizer(name_or_path='./finetune-hf-vits/shn-add-token', vocab_size=49, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '<unk>', 'pad_token': '|'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("|", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	49: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}


In [5]:
import torch.nn as nn

# print(model.text_encoder.embed_tokens)

class VitsModel(nn.Module):
    def __init__(self, model, tokenizer):
        super(VitsModel, self).__init__()
        self.model = model
        self.tokenizer = tokenizer
        
        # Assume `embeddings` is the original embedding layer in the VITS model
        old_embeddings = model.text_encoder.embed_tokens
        old_embedding_weight = old_embeddings.weight.data
        
        # Define new embedding layer with updated size
        new_embedding_layer = nn.Embedding(len(tokenizer) - 1, old_embedding_weight.shape[1])
        
        # Copy old weights into the new embedding layer
        new_embedding_layer.weight.data[:old_embedding_weight.size(0), :] = old_embedding_weight
        
        # Initialize new token embeddings (e.g., with the mean of existing ones)
        new_token_embeddings = old_embedding_weight.mean(dim=0, keepdim=True).repeat(len(new_tokens), 1)
        new_embedding_layer.weight.data[-len(new_tokens):, :] = new_token_embeddings
        
        # Replace the embedding layer in the model
        self.model.text_encoder.embed_tokens = new_embedding_layer
    
    def forward(self, input_ids, *args, **kwargs):
        # Use the modified embedding layer and pass through the model
        embeddings = self.model.text_encoder.embed_tokens(input_ids)
        outputs = self.model(input_ids_embeds=embeddings, *args, **kwargs)
        return outputs

# Create a new model instance with modified embeddings
VitsModel(model, tokenizer)

VitsModel(
  (model): VitsModel(
    (text_encoder): VitsTextEncoder(
      (embed_tokens): Embedding(49, 192)
      (encoder): VitsEncoder(
        (layers): ModuleList(
          (0-5): 6 x VitsEncoderLayer(
            (attention): VitsAttention(
              (k_proj): Linear(in_features=192, out_features=192, bias=True)
              (v_proj): Linear(in_features=192, out_features=192, bias=True)
              (q_proj): Linear(in_features=192, out_features=192, bias=True)
              (out_proj): Linear(in_features=192, out_features=192, bias=True)
            )
            (dropout): Dropout(p=0.1, inplace=False)
            (layer_norm): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
            (feed_forward): VitsFeedForward(
              (conv_1): Conv1d(192, 768, kernel_size=(3,), stride=(1,))
              (conv_2): Conv1d(768, 192, kernel_size=(3,), stride=(1,))
              (dropout): Dropout(p=0.1, inplace=False)
              (act_fn): ReLU()
            )
    

In [6]:
from transformers import VitsTokenizer

tokenizer = VitsTokenizer.from_pretrained("./Finetune/finetune-hf-vits/tokenizer/shn-add-token")
text = "ၾႃႉၾူၼ်ၵမ်ႇလမ်သွမ်ႇ ၸွမ်းၾင်ႇၼမ်ႉၾင်ႇၼွင်"

print(tokenizer.decode(tokenizer.encode(text)))

ၾႃႉၾူၼ်ၵမ်ႇလမ်သွမ်ႇ ၸွမ်းၾင်ႇၼမ်ႉၾင်ႇၼွင်


In [7]:
repo_name = "./Finetune/finetune-hf-vits/tokenizer/shn-embeddings-token-model"

model.save_pretrained(repo_name)
tokenizer.save_pretrained(repo_name)

('./finetune-hf-vits/shn-embeddings-token-model\\tokenizer_config.json',
 './finetune-hf-vits/shn-embeddings-token-model\\special_tokens_map.json',
 './finetune-hf-vits/shn-embeddings-token-model\\vocab.json',
 './finetune-hf-vits/shn-embeddings-token-model\\added_tokens.json')

In [10]:
from transformers import VitsModel

modified_model = VitsModel.from_pretrained(repo_name)
print(modified_model)

VitsModel(
  (text_encoder): VitsTextEncoder(
    (embed_tokens): Embedding(49, 192)
    (encoder): VitsEncoder(
      (layers): ModuleList(
        (0-5): 6 x VitsEncoderLayer(
          (attention): VitsAttention(
            (k_proj): Linear(in_features=192, out_features=192, bias=True)
            (v_proj): Linear(in_features=192, out_features=192, bias=True)
            (q_proj): Linear(in_features=192, out_features=192, bias=True)
            (out_proj): Linear(in_features=192, out_features=192, bias=True)
          )
          (dropout): Dropout(p=0.1, inplace=False)
          (layer_norm): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
          (feed_forward): VitsFeedForward(
            (conv_1): Conv1d(192, 768, kernel_size=(3,), stride=(1,))
            (conv_2): Conv1d(768, 192, kernel_size=(3,), stride=(1,))
            (dropout): Dropout(p=0.1, inplace=False)
            (act_fn): ReLU()
          )
          (final_layer_norm): LayerNorm((192,), eps=1e-05, eleme